In [1]:
""" 전체 feature에서 statin_drug 제외로 mace 예측 / statin 복용 환자 data 대상 """

' 전체 feature에서 statin_drug 제외로 mace 예측 / statin 복용 환자 data 대상 '

In [2]:
from torch import nn
import torch
from torch import tensor 
from torch.autograd import Variable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import torch
import itertools
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm.notebook import tqdm
from pathlib import Path
import os

total_out = Path(os.getcwd()+'/Statin_preprocess.csv')
total_data= pd.read_csv(total_out)
print(total_data.shape)
print(list(total_data.columns))

target ='mace'
unused_feat = 'statin_drug'

# mri feature가 시작하는 column의 index 구하기
# np.where의 결과값이 array에 들어가기 때문에 방금 계산해 넣어놓은 [0]번째 값을 가져온다.
start_features_index = np.where(total_data.columns.values == "group_after_detail_1")[0][0]
features = list(total_data.columns[start_features_index:])
features.remove(unused_feat)

print(len(features))
print(features)

Num_feat = len(features)

(122398, 27)
['interval_statin', 'mace', 'group_after_detail_1', 'statin_drug', 'AGE', 'SEX', 'HTN', 'ICDHTN', 'trt_ICD_DM', 'DM', 'BMI', 'WAIST', 'BP_HIGH', 'BP_LWST', 'BLDS', 'TOT_CHOLE', 'TRIGLYCERIDE', 'HDL_CHOLE', 'LDL_CHOLE', 'HMG', 'smoking', 'econo', 'drinking', 'HMG.1', 'LDL_CHOLE.1', 'current_ascvd', 'risk']
24
['group_after_detail_1', 'AGE', 'SEX', 'HTN', 'ICDHTN', 'trt_ICD_DM', 'DM', 'BMI', 'WAIST', 'BP_HIGH', 'BP_LWST', 'BLDS', 'TOT_CHOLE', 'TRIGLYCERIDE', 'HDL_CHOLE', 'LDL_CHOLE', 'HMG', 'smoking', 'econo', 'drinking', 'HMG.1', 'LDL_CHOLE.1', 'current_ascvd', 'risk']


In [3]:
# statin column
statin_data = total_data[total_data['statin_drug']==1]
print(statin_data.shape)

(32125, 27)


In [4]:
statin_data

,interval_statin,mace,group_after_detail_1,statin_drug,AGE,SEX,HTN,ICDHTN,trt_ICD_DM,DM,...,HDL_CHOLE,LDL_CHOLE,HMG,smoking,econo,drinking,HMG.1,LDL_CHOLE.1,current_ascvd,risk
6,1646,0,12,1,75,2,0,0,0,0,...,41,136.0,12.500000,0,3,1,12.500000,136.0,0,0.141912
10,1810,0,10,1,75,2,0,1,1,1,...,40,73.0,14.000000,0,1,1,14.000000,73.0,0,0.281002
12,1166,0,10,1,75,1,0,1,1,1,...,51,109.0,12.000000,0,2,1,12.000000,109.0,0,0.414875
23,1246,0,10,1,75,2,0,0,1,1,...,68,180.0,11.699219,0,3,1,11.699219,180.0,0,0.341367
29,1145,0,10,1,75,1,0,0,1,1,...,40,86.0,15.898438,0,1,1,15.898438,86.0,0,0.382987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122374,1763,0,10,1,60,2,0,1,0,0,...,51,88.0,14.099609,0,3,1,14.099609,88.0,0,0.027272
122375,1513,0,9,1,60,2,0,0,0,0,...,79,195.0,12.298828,0,3,1,12.298828,195.0,0,0.019180
122378,1543,0,10,1,60,2,0,1,0,0,...,56,159.0,14.500000,0,2,1,14.500000,159.0,0,0.038093
122384,1573,0,10,1,60,2,0,1,0,0,...,67,92.0,12.798828,0,3,1,12.798828,92.0,0,0.023437


In [5]:
def feature(Num_feat, clf, test_data_processed, features):
    importance =clf.feature_importances_
    #plt.plot(importance)
    #plt.show()
    labels_importance=importance.argsort()[::-1] # argsort : 오름차순 정렬

    importance_sort = np.sort(importance)[::-1]

    feat_name_sort=test_data_processed[features].columns[labels_importance]
    important_features = pd.DataFrame() 
    
    for i in range (Num_feat):
        feature = pd.DataFrame([[feat_name_sort[i],importance_sort[i]]], columns = ['feature name', 'ratio'])
        important_features=pd.concat([important_features,feature])

    return important_features.reset_index(drop=True)

In [6]:
# Augmented
import torch
import itertools
from sklearn.metrics import confusion_matrix
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm.notebook import tqdm


def find_bestpar(train_data, valid_data, test_data, features):
       
    """train data 생성"""
    X_train = train_data[features].values
    Y_train = train_data[target].values
    
    """valid data 생성"""
    X_valid = valid_data[features].values
    Y_valid = valid_data[target].values
    
    """test data 생성"""
    X_test = test_data[features].values
    Y_test = test_data[target].values
    
    print(len(X_train), len(X_valid), len(X_test))
    
    # Store maximum auc
    max_auc= 0
    # Store maximum hypterparameter set
    max_hy = []
    
    """
    # define hyperparameter space : learning rate, 
    n_ = [4,8,16]                              # 
    lr_ = [2e-2, 1e-2, 5e-3, 2e-3, 1e-3, 1e-4] # learning rate
    w_ = [0.01, 0.001, 0.0001]                 # weight decay
    g_ = [0.95, 0.99, 0.9]                     # scheduler params - gamma
    ss_ = [10, 20, 30]                         # scheduler params - step_size
    """
    # Orginal hyperparameter space 
    
    # define hyperparameter space (quick version)
    n_ = [4,16]
    lr_ = [2e-2,1e-3]
    w_ = [0.01,0.001]
    g_ = [0.95,0.99]
    ss_ = [10,30]
    
    all_ = [n_, lr_, w_, g_, ss_]
    h_space = [s for s in itertools.product(*all_)]
    
    print("start training")
    count=0
    for hy in tqdm(h_space):
        count = count + 1
        clf = TabNetClassifier(n_a = hy[0],
                                n_d = hy[0],
                                optimizer_params = dict(lr=hy[1], weight_decay=hy[2]),
                                scheduler_params={"step_size":hy[4], "gamma":hy[3]},
                                scheduler_fn=torch.optim.lr_scheduler.StepLR,
                                verbose=0)

        clf.fit(X_train, Y_train, eval_set=[(X_valid, Y_valid)], eval_metric=['auc'], max_epochs=200 , patience=20)
       
        preds_acc = clf.predict(X_test)
        preds_prob = clf.predict_proba(X_test)
        test_auc = roc_auc_score(y_score=preds_prob[:,1], y_true=Y_test)
        test_acc = accuracy_score(preds_acc, Y_test)
            
        print('Valid score: %2f'% clf.best_cost, 'Test AUC: %.3f%%'%test_auc, 'Test ACC: %.3f%%'%test_acc)
    
        if np.mean(test_auc)>max_auc:
            print("Find new maximum test AUC!!\n")
            max_hy = hy
            max_valid_score = clf.best_cost
            max_auc = test_auc
            max_acc = test_acc
    
    return clf, max_hy, max_valid_score, max_auc, max_acc, preds_prob

In [7]:
def preprocessing (total_data):
    total_data_processed= total_data.fillna(0).reset_index(drop=True)
    
    # frac: 전체 row 중 몇 %를 반환할 지 결정 -> frac=1을 설정해서 모든 데이터를 반환
    # random_state: 추후 이것과 동일한 샘플링을 재현하기 위함
    # sample: 데이터에서 임의의 샘플 선정 -> frac=1이면 전체 data의 순서만 임의로 바뀜
    total_data_processed = total_data_processed.sample(frac=1,random_state=2020).reset_index(drop=True)
    
    print("done preprocessing")
    return total_data_processed

statin_data = preprocessing(statin_data)
print(len(statin_data))

done preprocessing
32125


In [8]:
# mace 발병 유무에 따른 data 비율 분석
# 0 : 1 = 117622 : 4776
print(len(total_data))
print(len(total_data[total_data[target]==0]), len(total_data[total_data[target]==1]))

# 0 : 1 = 30501 : 1624 -> 비율 맞춰서 train, test data 나눠야 함
print(len(statin_data))
statin_mace_0 = statin_data[statin_data[target]==0] # statin 복용 o & mace 발병 x
statin_mace_1 = statin_data[statin_data[target]==1] # statin 복용 o & mace 발병 o
print(len(statin_mace_0), len(statin_mace_1))

122398
117622 4776
32125
30501 1624


In [9]:
# mace 값에 따라서 각각 train : valid : test = 6 : 2 : 2 로 나누도록
mace_0_train_cut = (int)(len(statin_mace_0)*0.6) #24400
mace_0_valid_cut = (int)(len(statin_mace_0)*0.8) #24400

statin_mace_0_train = statin_mace_0[:mace_0_train_cut]
statin_mace_0_valid = statin_mace_0[mace_0_train_cut:mace_0_valid_cut]
statin_mace_0_test = statin_mace_0[mace_0_valid_cut:]
print(len(statin_mace_0_train), len(statin_mace_0_valid), len(statin_mace_0_test))

mace_1_train_cut = (int)(len(statin_mace_1)*0.6) #1299
mace_1_valid_cut = (int)(len(statin_mace_1)*0.8) #1299

statin_mace_1_train = statin_mace_1[:mace_1_train_cut]
statin_mace_1_valid = statin_mace_1[mace_1_train_cut:mace_1_valid_cut]
statin_mace_1_test = statin_mace_1[mace_1_valid_cut:]
print(len(statin_mace_1_train), len(statin_mace_1_valid), len(statin_mace_1_test))

train_data = pd.concat([statin_mace_0_train, statin_mace_1_train])
valid_data = pd.concat([statin_mace_0_valid, statin_mace_1_valid])
test_data = pd.concat([statin_mace_0_test, statin_mace_1_test])

print(int(len(statin_data)*0.8), int(len(statin_data)*0.2))
print(len(train_data), len(valid_data), len(test_data))

18300 6100 6101
974 325 325
25700 6425
19274 6425 6426


In [10]:
train_data

,interval_statin,mace,group_after_detail_1,statin_drug,AGE,SEX,HTN,ICDHTN,trt_ICD_DM,DM,...,HDL_CHOLE,LDL_CHOLE,HMG,smoking,econo,drinking,HMG.1,LDL_CHOLE.1,current_ascvd,risk
0,1364,0,12,1,74,1,0,1,0,0,...,56,149.0,16.296875,0,3,1,16.296875,149.0,0,0.347138
1,1431,0,12,1,74,2,0,1,0,0,...,497,143.0,14.199219,0,2,1,14.199219,143.0,0,0.114869
3,1791,0,10,1,64,2,0,1,0,0,...,47,112.0,13.000000,0,4,1,13.000000,112.0,0,0.043556
4,1797,0,10,1,62,2,0,0,1,1,...,60,91.0,12.298828,0,4,1,12.298828,91.0,0,0.079931
5,1747,0,9,1,74,2,0,0,0,0,...,68,217.0,14.898438,0,2,1,14.898438,217.0,0,0.179291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19091,705,1,1,1,70,1,0,1,0,0,...,46,111.0,13.500000,1,2,8,13.500000,111.0,1,0.294097
19143,986,1,10,1,71,1,0,1,1,1,...,51,88.0,13.500000,0,4,1,13.500000,88.0,0,0.279536
19165,661,1,10,1,67,1,0,1,1,1,...,55,123.0,13.298828,0,2,2,13.298828,123.0,0,0.314191
19200,790,1,12,1,67,1,0,1,0,0,...,39,36.0,15.199219,1,1,2,15.199219,36.0,0,0.185375


In [11]:
class model():
    def __init__(self, train_data, valid_data, test_data, features):
        clf, max_hy, max_valid_score, max_test_auc, max_test_acc, preds_prob = find_bestpar(train_data, valid_data, test_data, features)    
    
        self.train_data = train_data
        self.test_data = test_data
        self.valid_data = valid_data
        self.preds_prob = preds_prob 
        self.max_hy = max_hy
        self.test_auc = max_test_auc
        self.test_acc = max_test_acc
        self.valid_score = max_valid_score
        self.clf = clf
        self.features = features

In [12]:
TabNet_mace_statin_only_only_total_feature_without_statin_drug_model = model(train_data, valid_data, test_data, features)

19274 6425 6426
start training



Early stopping occurred at epoch 32 with best_epoch = 12 and best_val_0_auc = 0.74086
Best weights from best epoch are automatically used!
Valid score: 0.740856 Test AUC: 0.722% Test ACC: 0.949%
Find new maximum test AUC!!


Early stopping occurred at epoch 37 with best_epoch = 17 and best_val_0_auc = 0.73773
Best weights from best epoch are automatically used!
Valid score: 0.737725 Test AUC: 0.729% Test ACC: 0.949%
Find new maximum test AUC!!


Early stopping occurred at epoch 52 with best_epoch = 32 and best_val_0_auc = 0.73668
Best weights from best epoch are automatically used!
Valid score: 0.736680 Test AUC: 0.728% Test ACC: 0.949%

Early stopping occurred at epoch 37 with best_epoch = 17 and best_val_0_auc = 0.73773
Best weights from best epoch are automatically used!
Valid score: 0.737725 Test AUC: 0.729% Test ACC: 0.949%

Early stopping occurred at epoch 93 with best_epoch = 73 and best_val_0_auc = 0.73868
Best weights from best epoch are automatically used!
Valid score: 0.738

In [14]:
print("<<Important Feature>>")
import_feat=feature(Num_feat, TabNet_mace_statin_only_only_total_feature_without_statin_drug_model.clf, TabNet_mace_statin_only_only_total_feature_without_statin_drug_model.test_data, TabNet_mace_statin_only_only_total_feature_without_statin_drug_model.features)
TabNet_mace_statin_only_only_total_feature_without_statin_drug_model.import_feat =  import_feat
import_feat

<<Important Feature>>


,feature name,ratio
0,trt_ICD_DM,0.100502
1,group_after_detail_1,0.079488
2,risk,0.070505
3,econo,0.063597
4,current_ascvd,0.061131
5,SEX,0.058329
6,AGE,0.055044
7,ICDHTN,0.053216
8,HTN,0.048306
9,WAIST,0.048055


In [15]:
import dill

with open('./TabNet_mace_statin_only_only_total_feature_without_statin_drug_model.pkl', 'wb') as f:
    dill.dump(TabNet_mace_statin_only_only_total_feature_without_statin_drug_model, f)

In [16]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score

"""
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
"""

# data 
# mace 값에 따라서 각각 train : test = 8 : 2 로 나누도록
mace_0_train_cut = (int)(len(statin_mace_0)*0.8) #24400

statin_mace_0_train = statin_mace_0[:mace_0_train_cut]
statin_mace_0_test = statin_mace_0[mace_0_train_cut:]
print(len(statin_mace_0_train), len(statin_mace_0_test))

mace_1_train_cut = (int)(len(statin_mace_1)*0.8) #1299

statin_mace_1_train = statin_mace_1[:mace_1_train_cut]
statin_mace_1_test = statin_mace_1[mace_1_train_cut:]
print(len(statin_mace_1_train), len(statin_mace_1_test))

train_data = pd.concat([statin_mace_0_train, statin_mace_1_train])
test_data = pd.concat([statin_mace_0_test, statin_mace_1_test])

print(int(len(statin_data)*0.8), int(len(statin_data)*0.2))
print(len(train_data), len(test_data))

X_train = train_data[features]
y_train = train_data[target]

X_test = test_data[features]
y_test = test_data[target]

# model 학습
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42, eval_metric="auc")
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)
y_pred_prob = xgb_model.predict_proba(X_test)[:,1]

print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred_prob))
#print(xgb_model.best_params_)

24400 6101
1299 325
25700 6425
25699 6426


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.9474011826953004
0.6662887546808216
